<h1>This notebook belongs to Liviu Petre and will be used for the capstone project for the "IBM Data Science Professional" specialization. Looking forward to it!</h1>

<h2 style="color:red">Beginning of question 1</h2>

In [1]:
#first import the needed libraries; only panda for now
import pandas as pd

<h3>In order to read the table, I will use the read_html method, and identify the first wikitable</h3>

In [2]:
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<br>
<h3>We will start now cleaning the data frame as specified. First, drop the lines where Boroughs are Not assigned</h3>

In [3]:
#clean up Borough = Not assigned
df.drop(df[df.Borough == "Not assigned"].index, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<br>
<h3>According to instructions: "If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.". We will check if there is any such case</h3>

In [4]:
# check any Not assigned  neighborhood
df[df.Neighbourhood == "Not assigned"]

,Postal Code,Borough,Neighbourhood


<h4>No such case exists!</h4>

<br>
<h3>Next we will check if more than one neighborhood can exist in one postal code area. For this we will first group by postal code and see how many rows we have and compare to the original data frame</h3>

In [5]:
#check if more than one neighborhood can exist in one postal code area
df[['Postal Code','Borough']].groupby(['Postal Code']).agg('count')

,Borough
Postal Code,
M1B,1
M1C,1
M1E,1
M1G,1
M1H,1
...,...
M9N,1
M9P,1
M9R,1


<h4>Grouping by postal code gives 103 rows. Let's now check the number of lines in the original data frame:</h4>

In [6]:
#check number of rows in data frame
df.shape

(103, 3)

<h4>So we have the same number of rows as in the grouping above, 103. This means each postal code appears once, so, as you can see from the wikipedia page, the neighbourhoods area already joined in the same boroughs.</h4>

<br>
<h3>We now reset the index and rename the columns exactly as the exercise requires </h3>

In [7]:
df.reset_index(drop=True, inplace=True)
df.columns=["PostalCode","Borough","Neighborhood"]
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<br>
<h3>And last, we call the shape method</h3> 

In [8]:
df.shape

(103, 3)

<h2 style="color:red">End of question 1</h2>

***

<h2 style="color:red">Beginning of question 2</h2>

<h3>First I tried using the geocoder library</h3>

In [10]:
#first install the package
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [11]:
#then import the pacakge
import geocoder

<br>
<h3>Now initiate the 2 additional columns of the data frame with float values</h3>

In [28]:
df['Latitude'] = len(df['PostalCode'])*[0.000000]
df['Longitude'] = len(df['PostalCode'])*[0.000000]
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0.0,0.0
1,M4A,North York,Victoria Village,0.0,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0.0,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0.0,0.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0.0,0.0
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",0.0,0.0
99,M4Y,Downtown Toronto,Church and Wellesley,0.0,0.0
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",0.0,0.0
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.0,0.0


<br>
<h3>Now we will iterate through each postal code, and for each, try to use geocoder to obtain the coordinates, then set them in the data frame</h3>

In [31]:
# iterate through the postal codes
for postal_code in df['PostalCode']:
    lat_lng_coords = None
    # loop until you get the coordinates
    while((lat_lng_coords is None)):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    # set latitude and longitude
    df.at[df.loc[df['PostalCode']==postal_code].index.tolist()[0],'Latitude'] = lat_lng_coords[0]
    df.at[df.loc[df['PostalCode']==postal_code].index.tolist()[0],'Longitude'] = lat_lng_coords[1]

KeyboardInterrupt: 

<h4 style="color:red">Unfortunately this did not work, as the geocoder library does not function properly, so I had to stop it</h4>

<br>
<h3> So, now, I will use the excel provided in the course. First I will import it in another data frame </h3>

In [17]:
url="http://cocl.us/Geospatial_data"
dfLatLong=pd.read_csv(url)

In [32]:
dfLatLong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<br>
<h3>Now, for each postal code, set the value in the initial data frame using the "at" method</h3>

In [33]:
for postal_code in df['PostalCode']:
    df.at[df.loc[df['PostalCode']==postal_code].index.tolist()[0],'Latitude'] = dfLatLong.at[dfLatLong.loc[dfLatLong['Postal Code']==postal_code].index.tolist()[0],'Latitude']
    df.at[df.loc[df['PostalCode']==postal_code].index.tolist()[0],'Longitude'] = dfLatLong.at[dfLatLong.loc[dfLatLong['Postal Code']==postal_code].index.tolist()[0],'Longitude']

<br>
<h3>Now the display the complete data frame</h3>

In [39]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h2 style="color:red">End of question 2</h2>

***

<h2 style="color:red">Beginning of question 3</h2>